In [1]:
import pandas as pd
import numpy as np
import re
import importlib
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
import sys
# Append the path to the parent directory of 'src'
# This assumes you're running the notebook in the 'notebook_folder' directory
sys.path.append('../')


In [19]:
import src.raw_data as raw_data
importlib.reload(raw_data)

<module 'src.raw_data' from 'c:\\Users\\felix\\Documents\\RAG\\notebooks\\..\\src\\raw_data.py'>

In [20]:
raw_data = raw_data.DataLoader(directory="C:/Users/felix/Documents/RAG/data/pdf")

In [21]:
corpus = raw_data.gen_corpus()

In [22]:
import src.preprocessing as preprocessing
importlib.reload(preprocessing)

preprocessor = preprocessing.Preprocessor(corpus=corpus)

In [23]:
preprocessor.clean_data()

In [24]:
corpus = preprocessor.corpus

In [25]:
corpus = [doc.replace("\n", " ") for doc in corpus]
corpus = [re.sub(r'\s+', ' ', doc) for doc in corpus]


In [26]:
print(corpus[0])

Angebot Privat-Haftpflichtversicherung | Felix Fikowski Referenz: 708/163349-E-00 | 2/1/1 Ihr gewünschter Schutz in der Privat-Haftpflichtversicherung bei der HUK-COBURG-Allgemeine Versicherung AG Privat-Haftpflichtversicherung Classic Für Singles 100 Millionen Euro Versicherungssumme für Personen-, Sach- und Vermögensschäden (bei Personenschäden maximal 15 Millionen Euro je geschädigte Person) Ohne Selbstbeteiligung mit Privat-Haftpflicht PLUS Erweitert den Schutz, zum Beispiel für Tageseltern, Drohnen bis 5 kg, Vermietung einer Eigentumswohnung Ihr Jahresbeitrag inklusive 19,00 % Versicherungssteuer 36,86 Euro 8,00 Euro 44,86 Euro Folgende Angaben sind in der Beitragsberechnung berücksichtigt Tarif Postleitzahl Geburtsdatum Vorschäden Nicht-öffentlicher Dienst 53359; Tarifzone 1 13.01.1996 Nein Hinweise: Dieses Angebot haben wir auf Grund Ihrer Angaben erstellt. Ändern sich die für die Beitragsberech­ nung maßgeblichen Angaben, kann sich der Beitrag ändern. Das Gleiche gilt, wenn sic

#### Preprocessing for the tf-idf Vectorizer

In [ ]:
corpus = [doc.replace("\n", " ") for doc in corpus]
corpus = [re.sub(r'\s+', ' ', doc) for doc in corpus]
corpus = [re.sub(r'(\w\s){3,}', ' ', doc) for doc in corpus]

In [28]:
preprocessed_corpus = []

for k in np.arange(len(corpus)):
    text= ""
    for i in np.arange(len(corpus[k])):
        if corpus[k][i].isalpha() or corpus[k][i].isspace():
            text += corpus[k][i].lower()
    text_split = text.split()
    preprocessed_corpus.append(text_split)

In [29]:
preprocessed_corpus[0]

['angebot',
 'privathaftpflichtversicherung',
 'felix',
 'fikowski',
 'referenz',
 'e',
 'ihr',
 'gewünschter',
 'schutz',
 'in',
 'der',
 'privathaftpflichtversicherung',
 'bei',
 'der',
 'hukcoburgallgemeine',
 'versicherung',
 'ag',
 'privathaftpflichtversicherung',
 'classic',
 'für',
 'singles',
 'millionen',
 'euro',
 'versicherungssumme',
 'für',
 'personen',
 'sach',
 'und',
 'vermögensschäden',
 'bei',
 'personenschäden',
 'maximal',
 'millionen',
 'euro',
 'je',
 'geschädigte',
 'person',
 'ohne',
 'selbstbeteiligung',
 'mit',
 'privathaftpflicht',
 'plus',
 'erweitert',
 'den',
 'schutz',
 'zum',
 'beispiel',
 'für',
 'tageseltern',
 'drohnen',
 'bis',
 'kg',
 'vermietung',
 'einer',
 'eigentumswohnung',
 'ihr',
 'jahresbeitrag',
 'inklusive',
 'versicherungssteuer',
 'euro',
 'euro',
 'euro',
 'folgende',
 'angaben',
 'sind',
 'in',
 'der',
 'beitragsberechnung',
 'berücksichtigt',
 'tarif',
 'postleitzahl',
 'geburtsdatum',
 'vorschäden',
 'nichtöffentlicher',
 'dienst',
 

#### Lemmatizer

In [34]:
import spacy

# Load the German NLP model
nlp = spacy.load("de_core_news_sm")


In [36]:
def lemmatize_doc(doc):
    # Join the words into a single string (since Spacy expects text input, not a list of tokens)
    text = " ".join(doc)
    # Process the text through the NLP pipeline
    processed_text = nlp(text)
    # Extract the lemmatized form of each word
    return [token.lemma_ for token in processed_text]

In [37]:
lemmatized_corpus = [lemmatize_doc(doc) for doc in preprocessed_corpus]

#### Stop Word Removal

In [39]:
pip install nltk

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\felix\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [40]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
from nltk.corpus import stopwords

german_stopwords = set(stopwords.words('german'))


In [42]:
def remove_stopwords(doc, stopwords):
    return [word for word in doc if word not in stopwords]

In [43]:
cleaned_corpus = [remove_stopwords(doc, german_stopwords) for doc in preprocessed_corpus]

In [44]:
cleaned_corpus[0]

['angebot',
 'privathaftpflichtversicherung',
 'felix',
 'fikowski',
 'referenz',
 'e',
 'gewünschter',
 'schutz',
 'privathaftpflichtversicherung',
 'hukcoburgallgemeine',
 'versicherung',
 'ag',
 'privathaftpflichtversicherung',
 'classic',
 'singles',
 'millionen',
 'euro',
 'versicherungssumme',
 'personen',
 'sach',
 'vermögensschäden',
 'personenschäden',
 'maximal',
 'millionen',
 'euro',
 'je',
 'geschädigte',
 'person',
 'selbstbeteiligung',
 'privathaftpflicht',
 'plus',
 'erweitert',
 'schutz',
 'beispiel',
 'tageseltern',
 'drohnen',
 'kg',
 'vermietung',
 'eigentumswohnung',
 'jahresbeitrag',
 'inklusive',
 'versicherungssteuer',
 'euro',
 'euro',
 'euro',
 'folgende',
 'angaben',
 'beitragsberechnung',
 'berücksichtigt',
 'tarif',
 'postleitzahl',
 'geburtsdatum',
 'vorschäden',
 'nichtöffentlicher',
 'dienst',
 'tarifzone',
 'nein',
 'hinweise',
 'angebot',
 'grund',
 'angaben',
 'erstellt',
 'ändern',
 'beitragsberech',
 'nung',
 'maßgeblichen',
 'angaben',
 'beitrag',


#### Tf-IDF Vectorizer

In [47]:
for i in np.arange(len(cleaned_corpus)):
    value = ' '.join(cleaned_corpus[i])
    cleaned_corpus[i] = value

In [45]:
vectorizer = TfidfVectorizer()

In [48]:
X = vectorizer.fit_transform(cleaned_corpus)

In [58]:
# Function to get top n words with highest TF-IDF scores in each document
def top_tfidf_words(tfidf_matrix, feature_names, top_n=7):
    top_words_corpus = []

    for doc_idx, doc in enumerate(tfidf_matrix):
        print(f"Document {doc_idx + 1}:")
        # Sort the TF-IDF values in descending order and get the indices
        sorted_indices = np.argsort(doc.toarray()).flatten()[::-1]
        # Get the top n words with the highest TF-IDF scores
        top_words = [feature_names[idx] for idx in sorted_indices[:top_n]]
        for word in top_words:
            print(f"{word}")

        top_words_corpus.append(top_words)
        print()  # Print a newline for better readability

    return top_words_corpus

In [59]:
# Get the feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Call the function to print top 3 words in each document
top_words_corpus = top_tfidf_words(X, feature_names, top_n=7)

Document 1:
privathaftpflichtversicherung
euro
angebot
millionen
telefon
maßgeblichen
angaben

Document 2:
glasversicherung
angebot
wohnung
telefon
mehrfamilienhaus
maßgeblichen
angaben

Document 3:
euro
hausratversicherung
angebot
fortsetzung
weils
selbstbeteiligung
wohnung

Document 4:
uhr
vermittlerin
hukcoburg
rheinbach
telefon
petra
tillmanns

Document 5:
uhr
vermittlerin
hukcoburg
telefon
petra
tillmanns
rheinbach

Document 6:
unterlagen
kopien
ablauf
zukommen
digital
eingereichte
gang

Document 7:
unterlagen
kopien
ablauf
zukommen
digital
eingereichte
gang

Document 8:
ständig
sachen
abweichend
bewohnte
wohnungen
versichert
versicherungswert

Document 9:
euro
uhr
privathaftpflichtversicherung
privat
antrag
telefon
deutschland

Document 10:
uhr
privat
antrag
glasversicherung
telefon
angaben
versicherungsschutz

Document 11:
euro
beispiel
schutz
antrag
inklusive
naturgefahren
uhr

Document 12:
unternehmensgruppe
hukcoburg
gmbh
einwilligung
sowie
daten
gesellschaften

Document 13:


In [60]:
top_words_corpus

[['privathaftpflichtversicherung',
  'euro',
  'angebot',
  'millionen',
  'telefon',
  'maßgeblichen',
  'angaben'],
 ['glasversicherung',
  'angebot',
  'wohnung',
  'telefon',
  'mehrfamilienhaus',
  'maßgeblichen',
  'angaben'],
 ['euro',
  'hausratversicherung',
  'angebot',
  'fortsetzung',
  'weils',
  'selbstbeteiligung',
  'wohnung'],
 ['uhr',
  'vermittlerin',
  'hukcoburg',
  'rheinbach',
  'telefon',
  'petra',
  'tillmanns'],
 ['uhr',
  'vermittlerin',
  'hukcoburg',
  'telefon',
  'petra',
  'tillmanns',
  'rheinbach'],
 ['unterlagen',
  'kopien',
  'ablauf',
  'zukommen',
  'digital',
  'eingereichte',
  'gang'],
 ['unterlagen',
  'kopien',
  'ablauf',
  'zukommen',
  'digital',
  'eingereichte',
  'gang'],
 ['ständig',
  'sachen',
  'abweichend',
  'bewohnte',
  'wohnungen',
  'versichert',
  'versicherungswert'],
 ['euro',
  'uhr',
  'privathaftpflichtversicherung',
  'privat',
  'antrag',
  'telefon',
  'deutschland'],
 ['uhr',
  'privat',
  'antrag',
  'glasversicher